In [6]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium import webdriver
import datetime
import urllib
import os
import mysql.connector
from sqlalchemy import create_engine

user = os.environ['MYSQL_USER']
host = os.environ['MYSQL_HOST']
pw = os.environ['MYSQL_PASSWORD']
db = os.environ['MYSQL_DATABASE']

user = 'root'
host = 'localhost'
pw = ''
db = 'heatseek'

conn_str = "mysql+mysqlconnector://{0}:{1}@{2}/{3}".format(user, pw, host, db)
engine = create_engine(conn_str, echo=False)
PHANTOM_PATH = '/Applications/anaconda/anaconda/bin/phantomJS' ## bash% which phantomJS

dcap = dict(DesiredCapabilities.PHANTOMJS)
dcap["phantomjs.page.settings.userAgent"] = ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36")
driver = webdriver.PhantomJS(executable_path=PHANTOM_PATH, desired_capabilities=dcap)
driver.set_window_size(1024, 768) 
driver.get('http://landlordwatchlist.com/search.html')

soup = BeautifulSoup(driver.page_source, "lxml")

result_dict = []
for i in soup.findAll("li"):
    try: 
        j,k = i.find("a", {"class":"js--is-toggleable-item"}).text.split("  Landlord: ")
        address = j[18:-6]
        landlord = k.split("Landlord Rank: ")[0]
    except: 
        address = '' 
        landlord = ''
    try: hpd_violations = i.find("div").text[26:].split("N")[0]
    except: hpd_violations = ''
    try: dob_violations = i.find("div").text[26:].split("N")[1][24:]
    except: dob_violations = ''
    result_dict.append([address,landlord,hpd_violations,dob_violations])

df = pd.DataFrame(result_dict, columns=['ADDR','LANDLORD','COUNT_HPD_VIO','COUNT_DOB_VIO'])
df = df[df['COUNT_HPD_VIO'] != '']
df['COUNT_HPD_VIO'] = df['COUNT_HPD_VIO'].astype(int)
df['COUNT_DOB_VIO'] = df['COUNT_DOB_VIO'].astype(int)


df.to_sql(name='pubadv_worst_landlords', con=engine, if_exists = 'replace', index=False, chunksize=2500)